In [17]:
# pip install google-generativeai
# pip install rouge

import google.generativeai as genai
import json
import os

from dotenv import load_dotenv
from rouge import Rouge

In [12]:
load_dotenv()
genai.configure(api_key=os.environ['GEMINI_KEY'])
directory_path = os.environ['DATA_PATH']

cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"

datasetRaw = {}

for file_name in [ocred_fn, cleaned_fn]:
    if file_name not in os.listdir(directory_path):
        print(f"ERROR 404 ! File {file_name} not Found...")

    file_path = os.path.join(directory_path, file_name)
    with open(file_path, 'r') as file:
        datasetRaw[file_name.split('.')[0]] = json.load(file)
        file.close()


In [36]:
def segment_wrapper(segment_ocred, segment_clean):
    return f"""
        ### Task: Il primo testo è una versione del secondo estratto con oc-Red, ed è stata processata per ridurre gli errori, valuta con un punteggio tra 1 e 5 la somglianza tra le due.
        ### Testo da valutare: {segment_ocred}.
        ### Testo di confronto: {segment_clean}.
        ### Risultato:
    """

In [28]:
sample_in = datasetRaw['original_ocr']['1']
sample_out = datasetRaw['cleaned']['1']

In [37]:

prompt = segment_wrapper(sample_in, sample_out)

In [38]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
hypothesis = model.generate_content(prompt).text
print(hypothesis)

Il testo da valutare presenta numerose differenze rispetto al testo di confronto,  principalmente dovute a errori ortografici, di punteggiatura e di trascrizione nel testo I.  Ci sono anche alcune piccole differenze nella formulazione di alcune frasi.  Considerando la presenza di questi errori e le modifiche necessarie per allineare il testo I al testo di confronto, assegno un punteggio di **3**.

Mentre la trama e la sequenza degli eventi sono le stesse, la qualità della scrittura e l'accuratezza linguistica del testo I sono inferiori, richiedendo una significativa revisione per raggiungere la parità con il testo di confronto.



In [ ]:
scorer = Rouge()
score = scorer.get_scores(hyps=sample_in, refs=sample_out)

In [ ]:
metrics_list = ['rouge-1', 'rouge-2', 'rouge-l']
for metric in metrics_list:
    print(metric," -> ",score[0][metric])

{'r': 0.8852459016393442, 'p': 0.8223350253807107, 'f': 0.8526315739541551}
{'r': 0.8287112561174551, 'p': 0.7925117004680188, 'f': 0.8102073315256189}
{'r': 0.8852459016393442, 'p': 0.8223350253807107, 'f': 0.8526315739541551}
